# Things left to do:
    
    1. 10 top genres ✓
    2. msPlayed*each feature, then get avg
    3. Last step: use top genres and top artists to select from knn output, take repetitive ones out
    
    4. merge multiple user data files function ✓

In [1]:
#import time
#import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import tkinter as tk
from tkinter import filedialog
%config InlineBackend.figure_format ='retina'
import random
from functools import reduce
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2

In [3]:
#####CHANGE TO IMPORT YOUR FILE(s) ONLY######
jake1 = pd.read_json('Jake Streaming History1.json')
jake2 = pd.read_json('Jake Streaming History2.json')
jake3 = pd.read_json('Jake Streaming History3.json')

In [4]:
def get_100_favorite_artists(filename):
    artists = {}
    stream = pd.read_json(filename)

    for j in stream['artistName']:
        if j not in artists:
            artists[j] = 1
        elif j in artists:
            artists[j] += 1
    art_lists = sorted(artists.items(),key = lambda kv: kv[1],reverse = True)
    
    artist_list = []
    for i in art_lists[:100]:
        artist_list.append(i[0])
        
    return artist_list

In [5]:
####CHANGE TO YOUR DATA####
hannah_art = get_100_favorite_artists('Jake Streaming History1.json')

In [6]:
#########SEE 2 CELLS ABOVE##########
jake1

,endTime,artistName,trackName,msPlayed
0,2021-04-03 07:52,The Beatles,Let It Be - Remastered 2009,29376
1,2021-04-03 07:52,The Beatles,Let It Be - Remastered 2009,1536
2,2021-04-03 07:59,The Beatles,Let It Be - Remastered 2015,20032
3,2021-04-03 08:07,League of Legends,Space Groove - 2021,22058
4,2021-04-03 18:57,NOTHING MORE,Sex & Lies,1962
...,...,...,...,...
9995,2021-09-22 19:54,The Killers,For Reasons Unknown,212266
9996,2021-09-22 19:58,The Killers,Read My Mind,246840
9997,2021-09-22 20:02,The Killers,Uncle Jonny,265800
9998,2021-09-22 20:06,The Killers,Bones,227000


In [7]:
###########CHANGE hannah TO WHAT YOU DEFINED EARLIER##########
jake1.sort_values('msPlayed', ascending = False).head(100)

,endTime,artistName,trackName,msPlayed
3626,2021-06-09 17:59,EUphoria Podcast,"New Team, Who Dis? (ft. LIDER) | EUphoria | 20...",4667386
7009,2021-08-12 19:05,EUphoria Podcast,All About Playoffs (ft. Vedius) | EUphoria | 2...,2896388
6566,2021-08-07 16:09,The Daily,The End of Andrew Cuomo?,2343332
3806,2021-06-13 06:43,C418,The End,904528
9746,2021-09-20 22:08,Pink Floyd,Shine On You Crazy Diamond (Pts. 1-5),811077
...,...,...,...,...
6909,2021-08-11 22:13,Gustav Holst,"The Planets, Op. 32: 4. Jupiter, the Bringer o...",469760
9705,2021-09-20 19:43,Pink Floyd,Us and Them,469224
9473,2021-09-18 00:57,Pink Floyd,Us and Them,469224
9567,2021-09-19 00:23,Pink Floyd,Us and Them,469224


In [8]:
def get_track_name_time_artist(filename):
    tracks = {}
    stream = pd.read_json(filename)

    for i in range(len(stream)):
        name = stream['trackName'][i]
        time = stream['msPlayed'][i]
        artist = stream['artistName'][i]
        if name not in tracks:
            tracks[name] = [time,artist]
        elif name in tracks:
            tracks[name][0] += time
    tracks = sorted(tracks.items(),key = lambda kv: kv[1],reverse = True)
    df = pd.DataFrame(tracks, columns =['track_names','time_artist'])

        
    return tracks,df

In [9]:
######### CHANGE THE STRING PART TO YOUR OWN DATA ############
tracks,tracks_time = get_track_name_time_artist('Jake Streaming History1.json')

In [10]:
#cut about 700 out from the bottom, to speed up
tracks_time = tracks_time.head(2000)
tracks_time

,track_names,time_artist
0,Aria Math,"[13832524, C418]"
1,Empire Ants (feat. Little Dragon),"[9855911, Gorillaz]"
2,Age of Machine,"[9343697, Greta Van Fleet]"
3,Voyager,"[9148162, Daft Punk]"
4,The Weight of Dreams,"[8887602, Greta Van Fleet]"
...,...,...
1995,The World Was Wide Enough,"[107435, Leslie Odom Jr.]"
1996,Break it Down,"[107026, アトラスサウンドチーム]"
1997,Like A Boss,"[106813, The Lonely Island]"
1998,Sex & Super Smash Bros.,"[106416, KYLE]"


In [11]:
all_songs = pd.read_csv('tracks.csv')
all_songs = all_songs.drop(columns = ['duration_ms','explicit','id','id_artists','mode'])
all_songs = all_songs.drop_duplicates(subset = ['name', 'artists'],
                                     keep = 'last').reset_index(drop = True)

In [12]:
final_all_songs = all_songs.sort_values('popularity', ascending = False)

In [13]:
names = tracks_time['track_names'].values.tolist()

In [14]:
#takes about 5 min to search for 900 songs (see # of songs below tracks_time)

###############CHANGE hannah_all TO YourName_all###############

jake1_all = pd.DataFrame()
for i in range(len(names)):
    name = names[i]
    artist = tracks_time['time_artist'][i][1]
    formatted = '[\''+artist+'\']'
    
    mask1 = final_all_songs['name'] == name
    mask2 = final_all_songs['artists'] == formatted
    jake1_all = jake1_all.append(final_all_songs[mask1&mask2])

In [15]:
############CHANGE TO WHAT YOU DEFINED IN THE PREV. CELL#################
jake1_all

,name,popularity,artists,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
57335,Voyager,67,['Daft Punk'],2001-03-12,0.669,0.743,7,-11.488,0.0494,0.282000,0.803000,0.3090,0.2790,119.882,4
52169,Nutshell,71,['Alice In Chains'],1994,0.384,0.561,3,-10.059,0.0296,0.373000,0.253000,0.0775,0.1590,136.136,3
446070,Cult of Personality,27,['Living Colour'],1988,0.508,0.875,0,-4.757,0.0435,0.000432,0.001030,0.3870,0.5350,92.194,4
41228,Life's Been Good,70,['Joe Walsh'],1978-06-01,0.505,0.476,2,-12.897,0.0711,0.250000,0.009150,0.1130,0.5420,99.310,4
93610,Aqueous Transmission,56,['Incubus'],2001-10-23,0.415,0.456,9,-12.082,0.0352,0.005010,0.763000,0.1780,0.0935,151.937,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101833,No Way Back,57,['Foo Fighters'],2005-06-14,0.254,0.959,9,-3.447,0.0672,0.000013,0.000007,0.4830,0.5250,186.438,4
41973,The Happiest Days of Our Lives,60,['Pink Floyd'],1979-11-30,0.741,0.231,9,-18.910,0.0415,0.551000,0.747000,0.2490,0.5470,103.994,4
35818,Since I've Been Loving You - Remaster,44,['Led Zeppelin'],1970-10-05,0.393,0.364,5,-10.945,0.0345,0.002100,0.070300,0.1210,0.3080,118.566,3
32658,Love You To - Remastered 2009,53,['The Beatles'],1966-08-05,0.391,0.593,5,-8.999,0.0413,0.700000,0.000249,0.0829,0.7070,123.986,4


In [16]:
####### DO LATER: EACH FEATURE TIMES BY TIME LISTENED, THEN AVERAGE ##########
print(jake1_all.describe().loc['mean'])
print(jake1_all.describe().loc['mean'].values.tolist())
Center = jake1_all.describe().loc['mean'].values.tolist()
del Center[0] #pop
del Center[2] #key
#del Center[8] #tempo, if uncomment, have to change below to 8 too.
del Center[9] #time_sig
print(Center)

popularity           56.220907
danceability          0.530209
energy                0.700468
key                   5.272189
loudness             -7.554249
speechiness           0.068825
acousticness          0.160234
instrumentalness      0.102520
liveness              0.210236
valence               0.517317
tempo               124.286253
time_signature        3.926036
Name: mean, dtype: float64
[56.22090729783037, 0.530209072978303, 0.7004682544378703, 5.272189349112426, -7.554248520710058, 0.06882475345167652, 0.160234423639053, 0.1025201913905325, 0.21023589743589738, 0.5173169625246541, 124.28625345167664, 3.92603550295858]
[0.530209072978303, 0.7004682544378703, -7.554248520710058, 0.06882475345167652, 0.160234423639053, 0.1025201913905325, 0.21023589743589738, 0.5173169625246541, 124.28625345167664]


In [17]:
# use: 
#     dance
#     energy
#     loudness
#     speech
#     acoust
#     instru
#     live
#     valen

# optional:
#     popularity
#     tempo

# do not use:
#     key
#     time_sig

In [18]:
def set_up_nineD(df):
    N = len(df)
    nineD = np.zeros([N,9])
    for i in range(N):
        nineD[i,0] = df['danceability'][i]
        nineD[i,1] = df['energy'][i]
        nineD[i,2] = df['loudness'][i]
        nineD[i,3] = df['speechiness'][i]
        nineD[i,4] = df['acousticness'][i]
        nineD[i,5] = df['instrumentalness'][i]
        nineD[i,6] = df['liveness'][i]
        nineD[i,7] = df['valence'][i]
        nineD[i,8] = df['tempo'][i]
    return nineD

In [19]:
from math import *
# Function for computing the distance between two points
def distance(p1,p2):
    '''Returns the distance betwen two points'''
    return sqrt((p1[0] - p2[0])**2 + 
                (p1[1] - p2[1])**2 +
                (p1[2] - p2[2])**2 +
                (p1[3] - p2[3])**2 +
                (p1[4] - p2[4])**2 +
                (p1[5] - p2[5])**2 +
                (p1[6] - p2[6])**2 +
                (p1[7] - p2[7])**2 +
                (p1[8] - p2[8])**2)

In [20]:
#k nearest neighbors
def knn (Center,nineD,k=3):
    N = len(nineD)
    dist_list = []
    for i in range(N):
        dist = distance(Center,nineD[i])
        dist_list.append(dist)
    
    min_value = []
    min_index = []
    for j in range(k):
        min_val = sorted(dist_list)[j]
        min_value.append(min_val)
        
        min_ind = dist_list.index(min_val)
        min_index.append(min_ind)
    return min_value , min_index

In [21]:
# #selecting top () popular songs from all
small_all = final_all_songs.head(100000)
small_all = small_all.reset_index()

In [22]:
nineD = set_up_nineD(small_all)

In [23]:
#called ok for fun
ok = 10
min_value , min_index = knn(Center,nineD,k=ok)

In [24]:
min_index

[9, 9842, 83648, 16647, 16758, 74422, 21972, 12690, 85762, 80961]

In [25]:
recommendation_df = small_all.loc[[9, 9842, 83648, 16647, 16758, 74422, 21972, 12690, 85762, 80961]]
recommendation_df = recommendation_df.reset_index()
recommendation_df

,level_0,index,name,popularity,artists,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,9,70817,Paradise (feat. Dermot Kennedy),92,"['MEDUZA', 'Dermot Kennedy']",2020-10-30,0.632,0.595,8,-7.644,0.0401,0.0689,0.000000,0.2090,0.435,124.114,4
1,9842,153393,i miss u (with Au/Ra),65,"['Jax Jones', 'Au/Ra']",2020-10-09,0.671,0.722,9,-7.550,0.0460,0.0469,0.000000,0.1180,0.404,124.096,4
2,83648,44291,Take A Chance With Me,46,['Roxy Music'],1982-05-01,0.604,0.670,10,-7.754,0.0274,0.0151,0.020400,0.2280,0.574,124.104,4
3,16647,330790,No One Knows (feat. Chiara Castelli),61,"['Henri PFR', 'Famba', 'Chiara Castelli']",2020-11-02,0.725,0.767,11,-7.472,0.0501,0.1140,0.000005,0.0954,0.447,124.103,4
4,16758,56731,American Pie,61,['Madonna'],2000-09-19,0.631,0.734,5,-7.480,0.0360,0.3480,0.000000,0.1350,0.591,124.036,4
5,74422,44982,Mi Niña Veneno (Menina Veneno) - Spanish,47,['Ritchie'],1983-01-01,0.621,0.749,11,-7.376,0.0313,0.2720,0.000045,0.2910,0.754,124.444,4
6,21972,91771,The Business - Vintage Culture & Dubdogz Remix,59,"['Tiësto', 'Vintage Culture', 'Dubdogz']",2021-04-09,0.760,0.692,8,-7.627,0.0579,0.0847,0.018600,0.3810,0.484,124.043,4
7,12690,36291,Baby Blue - Remastered 2010,63,['Badfinger'],1971-12-13,0.623,0.876,11,-7.788,0.0660,0.0189,0.013400,0.0728,0.515,124.431,4
8,85762,250360,Giati,45,['Melisses'],2018-07-18,0.576,0.394,7,-7.414,0.0573,0.1800,0.000001,0.1100,0.592,124.429,3
9,80961,192776,שבט אחים ואחיות,46,['שבט אחים ואחיות'],2019-04-18,0.598,0.631,9,-7.815,0.0371,0.2010,0.000000,0.1250,0.348,124.080,4


In [26]:
#see the "ok" above
for i in range(ok):
    print(recommendation_df['name'][i], recommendation_df['artists'][i],'\n')

Paradise (feat. Dermot Kennedy) ['MEDUZA', 'Dermot Kennedy'] 

i miss u (with Au/Ra) ['Jax Jones', 'Au/Ra'] 

Take A Chance With Me ['Roxy Music'] 

No One Knows (feat. Chiara Castelli) ['Henri PFR', 'Famba', 'Chiara Castelli'] 

American Pie ['Madonna'] 

Mi Niña Veneno (Menina Veneno) - Spanish ['Ritchie'] 

The Business - Vintage Culture & Dubdogz Remix ['Tiësto', 'Vintage Culture', 'Dubdogz'] 

Baby Blue - Remastered 2010 ['Badfinger'] 

Giati ['Melisses'] 

שבט אחים ואחיות ['שבט אחים ואחיות'] 



In [176]:
# client_id = '14498f1851cb44baa6af1641d5d1b650'
# client_secret = 'f68c94b6e4d445e9a005062e95e0be8d'

In [177]:
# import spotipy as sp
# from spotipy.oauth2 import SpotifyClientCredentials

# lz_uri = 'spotify:artist:36QJpDe2go2KgaRleHCDTp'

# spotify = sp.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id,client_secret))
# results = spotify.artist_top_tracks(lz_uri)

# for track in results['tracks'][:5]:
#     print('track    : ' + track['name'])
#     print('audio    : ' + track['preview_url'])
#     print('cover art: ' + track['album']['images'][0]['url'])
#     print()

In [178]:
# def get_artist(name):
#     results = spotify.search(q='artist:' + name, type='artist')
#     items = results['artists']['items']
#     if len(items) > 0:
#         return items[0]
#     else:
#         return None

In [179]:
# lz = get_artist('led zeppelin')
# lz

In [180]:
# top = []
# roco = 0
# for i in art_lists:
#     if roco < 6:
#         top.append(i[0])
#     roco += 1

# top

In [181]:
# reco = []
# for i in top:
#     reco.append(get_artist(i))
# #spotify.recommendation_genre_seeds()
# save = reco[4]['genres']
# save
# #spotify.

In [182]:
# root = tk.Tk()
# root.withdraw()

# files = filedialog.askopenfilenames()